<a href="https://colab.research.google.com/github/praveenprabharavindran/llama-index-n-fine-tuning/blob/main/llama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!add-apt-repository ppa:deadsnakes/ppa
!apt-get update
!apt-get install python3.10
!python3.10 --version



In [ ]:
!pip install llama-index
!pip install openai
!pip install requests
!pip install beautifulsoup4

In [ ]:
# Import necessary packages
from llama_index import GPTSimpleVectorIndex, Document, SimpleDirectoryReader, LLMPredictor, ServiceContext, PromptHelper
from langchain.chat_models import ChatOpenAI
from langchain import OpenAI

import os
import requests
from bs4 import BeautifulSoup

In [ ]:
os.environ['OPENAI_API_KEY'] = '<Enter api Key here'

In [ ]:
# Read text from target site
url = 'https://yaml.org/spec/1.2.2/'
response = requests.get(url)

soup = BeautifulSoup(response.text, 'html.parser')

# Find the <div> tag with class "content"
content_div = soup.find('div', {'class': 'container'})

if content_div:
    # Find all the <p> tags within the content_div and append their text to the 'text' variable
    text = ''
    for p in content_div.find_all('p'):
        text += p.text + '\n'
else:
    print('Content not found on the page')


In [ ]:
def construct_index_davinci_003(documents):
    max_input_size = 4096
    num_outputs = 2048
    max_chunk_overlap = 20
    chunk_size_limit = 600

    prompt_helper = PromptHelper(max_input_size, num_outputs, max_chunk_overlap, chunk_size_limit=chunk_size_limit)

    llm_predictor_gpt = LLMPredictor(llm=OpenAI(temperature=0.7, model_name="text-davinci-003", max_tokens=num_outputs))

      # index = GPTSimpleVectorIndex(documents, llm_predictor=llm_predictor_gpt, prompt_helper=prompt_helper)

    service_context = ServiceContext.from_defaults(
        llm_predictor=llm_predictor_gpt, prompt_helper=prompt_helper
    )

    index = GPTSimpleVectorIndex.from_documents(documents,service_context=service_context)

    # index.save_to_disk('index.json')

    return index

In [ ]:
def construct_index_gpt_35_turbo(documents):
    max_input_size = 4096
    num_outputs = 512
    max_chunk_overlap = 20
    chunk_size_limit = 600

    prompt_helper = PromptHelper(max_input_size, num_outputs, max_chunk_overlap, chunk_size_limit=chunk_size_limit)

    llm_predictor_gpt = LLMPredictor(llm=ChatOpenAI(temperature=0.7, model_name="gpt-3.5-turbo", max_tokens=num_outputs))

      # index = GPTSimpleVectorIndex(documents, llm_predictor=llm_predictor_gpt, prompt_helper=prompt_helper)

    service_context = ServiceContext.from_defaults(
        llm_predictor=llm_predictor_gpt, prompt_helper=prompt_helper
    )

    index = GPTSimpleVectorIndex.from_documents(documents,service_context=service_context)

    # index.save_to_disk('index.json')

    return index

In [ ]:
#Connect to open ai and build index
text_list = [text]
documents = [Document(t) for t in text_list]

# build index
#index = construct_index_davinci_003(documents)
index = construct_index_gpt_35_turbo(documents)

In [ ]:
# Querying the index
#response = index.query("Briefly summarize the history of yaml")
#response = index.query("What are explicit tags in yaml")
response = index.query("Suggest an outline for a 20 minute presentation on yaml based on the document")
print(response)